# Compute performance metrics for the given Y and Y_score without sklearn

In [1]:
import numpy as np
import pandas as pd
# other than these two you should not import any other packages

<pre>
<font color='red'><b>A.</b></font> Compute performance metrics for the given data <strong>5_a.csv</strong>
   <b>Note 1:</b> in this data you can see number of positive points >> number of negatives points
   <b>Note 2:</b> use pandas or numpy to read the data from <b>5_a.csv</b>
   <b>Note 3:</b> you need to derive the class labels from given score</pre> $y^{pred}= \text{[0 if y_score < 0.5 else 1]}$

<pre>
<ol>
<li> Compute Confusion Matrix </li>
<li> Compute F1 Score </li>
<li> Compute AUC Score</li>
<li> Compute Accuracy Score </li>
</ol>
</pre>

In [2]:
data=pd.read_csv("5_a.csv")
data.shape

(10100, 2)

In [3]:
def predict(df,y,thresh_hold):
    y_pred=[]
    for label in df[y]:
        if label<thresh_hold:
            y_pred.append(0)
        else:
            y_pred.append(1)
    return y_pred
    
    
# confusion matrix
def cal_vals(df,y,y_pred):
    tp=0
    tn=0
    fn=0
    fp=0
    for val1,val2 in enumerate(df['y']):
        if(df.y_pred[val1]==1) and df.y[val1]==1:
            tp=tp+1
        if(df.y_pred[val1]==0) and df.y[val1]==0:
            tn=tn+1
        if(df.y_pred[val1]==0) and df.y[val1]==1:
            fn=fn+1
        if(df.y_pred[val1]==1) and df.y[val1]==0:
            fp=fp+1
    return {'tn':tn,'tp':tp,'fn':fn,'fp':fp}

In [4]:
thresh_hold=0.5
data['y_pred']=predict(data,'proba',thresh_hold)
confusion_matrix=cal_vals(data,'y','y_pred')

In [5]:
# confusion matrix values 
print("the confusion matrix is: ",confusion_matrix)

the confusion matrix is:  {'tn': 0, 'tp': 10000, 'fn': 0, 'fp': 100}


In [6]:
# F1 score
x=data.y.value_counts()
P=x[1]

precision=confusion_matrix['tp']/(confusion_matrix['tp']+confusion_matrix['fp'])
recall=confusion_matrix['tp']/P

F1=2*precision*recall/(precision+recall)
print('the F1 score is: ',F1)


the F1 score is:  0.9950248756218906


In [7]:
# Accuracy
Acc=(confusion_matrix['tp']+confusion_matrix['tn'])/data.shape[0]
print('the accuracy is: ',Acc)


the accuracy is:  0.9900990099009901


In [8]:
# AUC score funtion
from tqdm import tqdm_notebook      # purpose of import is to just see progress
def auc(df):
    s = df['y'].value_counts()
    P = s[1]
    N = s[0]
    tpr = []
    fpr = []
    for elem in tqdm_notebook(df['proba']):
        df['y_pred']=predict(df,'proba',elem)
        confusion_matrix=cal_vals(df,'y','y_pred')
        tpr.append(confusion_matrix['tp']/P)
        fpr.append(confusion_matrix['fp']/N)
        df.drop(columns=['y_pred'])
    return np.trapz(tpr,fpr)

In [9]:
data=data.sort_values(by='proba',ascending=False)
data.drop(columns=['y_pred'])

,y,proba
1664,1.0,0.899965
2099,1.0,0.899828
1028,1.0,0.899825
9592,1.0,0.899812
8324,1.0,0.899768
...,...,...
8294,1.0,0.500081
1630,1.0,0.500058
7421,1.0,0.500058
805,1.0,0.500047


In [10]:
AUC_score=auc(data)
print ('the AUC Score is :',AUC_score)


the AUC Score is : 0.48829900000000004


<pre>
<font color='red'><b>B.</b></font> Compute performance metrics for the given data <strong>5_b.csv</strong>
   <b>Note 1:</b> in this data you can see number of positive points << number of negatives points
   <b>Note 2:</b> use pandas or numpy to read the data from <b>5_b.csv</b>
   <b>Note 3:</b> you need to derive the class labels from given score</pre> $y^{pred}= \text{[0 if y_score < 0.5 else 1]}$

<pre>
<ol>
<li> Compute Confusion Matrix </li>
<li> Compute F1 Score </li>
<li> Compute AUC Score </li>
</ol>
</pre>

In [11]:
data_B=pd.read_csv('5_b.csv')
data_B.shape

(10100, 2)

In [12]:
thresh_hold=0.5
data_B['y_pred']=predict(data_B,'proba',thresh_hold)
confusion_matrix_B=cal_vals(data_B,'y','y_pred')

In [13]:
# confusion matrix values 
print('the confusion matrix is :', confusion_matrix_B)

the confusion matrix is : {'tn': 9761, 'tp': 55, 'fn': 45, 'fp': 239}


In [14]:
# F1 score
x=data_B.y.value_counts()
P=x[1]

precision_B=confusion_matrix_B['tp']/(confusion_matrix_B['tp']+confusion_matrix_B['fp'])
recall_B=confusion_matrix_B['tp']/P

F1_B=2*precision_B*recall_B/(precision_B+recall_B)
print('the F1 Score is : ',F1_B)


the F1 Score is :  0.2791878172588833


In [15]:
# Accuracy
Acc_B=(confusion_matrix_B['tp']+confusion_matrix_B['tn'])/data_B.shape[0]
print('the Accuracy is :',Acc_B)

the Accuracy is : 0.9718811881188119


In [16]:
#AUC score
data_B=data_B.sort_values(by='proba',ascending=False)
data_B.drop(columns=['y_pred'])
AUC_score_B=auc(data_B)
print('the AUC Score is: ',AUC_score_B)


the AUC Score is:  0.9377570000000001


<font color='red'><b>C.</b></font> Compute the best threshold (similarly to ROC curve computation) of probability which gives lowest values of metric <b>A</b> for the given data <strong>5_c.csv</strong>
<br>

you will be predicting label of a data points like this: $y^{pred}= \text{[0 if y_score < threshold  else 1]}$

$ A = 500 \times \text{number of false negative} + 100 \times \text{numebr of false positive}$

<pre>
   <b>Note 1:</b> in this data you can see number of negative points > number of positive points
   <b>Note 2:</b> use pandas or numpy to read the data from <b>5_c.csv</b>
</pre>

In [17]:
# min metric function

def min_metric(data):
    s = data['y'].value_counts()
    P = s[1]
    N = s[0]
    tpr = []
    fpr = []
    metric={}
    for elem in tqdm_notebook(data['prob']):
        data['y_pred']=predict(data,'prob',elem)
        confusion_matrix=cal_vals(data,'y','y_pred')
#         tpr.append(confusion_matrix['tp']/P)
#         fpr.append(confusion_matrix['fp']/N)
        metric_val=(500*confusion_matrix['fn'])+(100*confusion_matrix['fp'])
        metric[elem]=metric_val
        data.drop(columns=['y_pred'])
    return(metric)

In [18]:
data=pd.read_csv('5_c.csv')
print(data.head())
print(data.shape)
data=data.sort_values(by='prob',ascending=False)
result=min_metric(data)


   y      prob
0  0  0.458521
1  0  0.505037
2  0  0.418652
3  0  0.412057
4  0  0.375579
(2852, 2)


In [19]:
temp = min(result.values()) 
res = [key for key in result if result[key] == temp]
print('the key:value pair for min value of the specified metric is-',res,temp)

the key:value pair for min value of the specified metric is- [0.2300390278970873] 141000


<pre>
<font color='red'><b>D.</b></font> Compute performance metrics(for regression) for the given data <strong>5_d.csv</strong>
    <b>Note 2:</b> use pandas or numpy to read the data from <b>5_d.csv</b>
    <b>Note 1:</b> <b>5_d.csv</b> will having two columns Y and predicted_Y both are real valued features
<ol>
<li> Compute Mean Square Error </li>
<li> Compute MAPE: https://www.youtube.com/watch?v=ly6ztgIkUxk</li>
<li> Compute R^2 error: https://en.wikipedia.org/wiki/Coefficient_of_determination#Definitions </li>
</ol>
</pre>

In [20]:
data_d=pd.read_csv("5_d.csv")
data_d.shape
data_d.head()

,y,pred
0,101.0,100.0
1,120.0,100.0
2,131.0,113.0
3,164.0,125.0
4,154.0,152.0


In [21]:
 
def error(df,col1,col2):
    val=[]
    for index, (value1, value2) in enumerate(zip(df[col1], df[col2])):
        val.append(value1-value2)
    return val
    
def absolute_error(df,col):
    val=[]
    for index,value in enumerate(df[col]):
        val.append(abs(value))
    return val

def mean_sq_error(df,col):
    return ss_res(df,col)/len(df[col])

def mape(df,col1,col2):
    val=sum(df[col1])/sum(df[col2])
    return val

def ss_res(df,col):
    val=0
    for index,value in enumerate(df[col]):
        val=val+(value*value)
    return val

def ss_tot(df,col):
    val=0
    mean_val=data_d['y'].mean()
    for index,value in enumerate(df[col]):
        val=val+ (value-mean_val)*(value-mean_val)
    return val
        
        

In [22]:
data_d['error']=error(data_d,'y','pred')
data_d['abs_error']=absolute_error(data_d,'error')

In [23]:
MSE=mean_sq_error(data_d,'error')
print("the Mean squared error is : ", MSE)

the Mean squared error is :  177.16569974554707


In [24]:
MAPE=mape(data_d,'abs_error','y')
print('the MAPE value is :', MAPE)

the MAPE value is : 0.1291202994009687


In [25]:
SS_RES=ss_res(data_d,'error')
SS_TOT=ss_tot(data_d,'y')
R_square= 1- (SS_RES/SS_TOT)
print('the Co-efficient of determination value is: ',R_square)

the Co-efficient of determination value is:  0.9563582786990964
